In [3]:
from torch_snippets import *
import torchvision 
from torchvision import transforms 
import torchvision.utils as vutils
import cv2, numpy as np, pandas as pd
import glob
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
!mkdir cropped_faces
images = Glob('/content/females/*.jpg') + glob('/content/males/*.jpg')
for i in range(len(images)
